In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bb1e92be41691402fa2a1adc833b2064de5c3ea0a3bcc8bb50e7cc1dce021b5a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    return scorer.score(reference, candidate)

In [ ]:
!pip install datasets
!pip install transformers
!pip install torch
!pip install nltk
!pip install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import re
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import nltk
from nltk.tokenize import sent_tokenize
from collections import Counter

nltk.download("punkt", quiet=True)




True

In [ ]:
nltk.download("punkt", quiet=True)

# --- Constants ---
ABSTRACTIVE_MODEL_NAME = "facebook/bart-large-cnn"
MAX_SAMPLES = 100
SUMMARY_MAX_LENGTH = 150
#added more n topics
N_TOPICS = 4


In [ ]:
# --- Enhanced Medical Stop Words ---
MEDICAL_STOP_WORDS = {
    "patient",
    "patients",
    "study",
    "studies",
    "result",
    "results",
    "method",
    "methods",
    "analysis",
    "group",
    "groups",
    "data",
    "showed",
    "show",
    "significant",
    "difference",
    "clinical",
    "the",
    "of",
    "and",
    "in",
    "with",
    "to",
    "was",
    "were",
    "for",
    "than",
    "or",
}

In [ ]:
# --- Model Initialization ---
def init_summarizer():
    print("Loading summarization model...")
    device = 0 if torch.cuda.is_available() else -1
    try:
        tokenizer = AutoTokenizer.from_pretrained(ABSTRACTIVE_MODEL_NAME)
        model = AutoModelForSeq2SeqLM.from_pretrained(ABSTRACTIVE_MODEL_NAME)
        summarizer = pipeline(
            "summarization", model=model, tokenizer=tokenizer, device=device
        )
        print(f"Running on {'GPU' if device == 0 else 'CPU'}")
        return summarizer
    except Exception as e:
        print(f"Model loading failed: {e}. Using extractive only.")
        return None


summarizer = init_summarizer()

Loading summarization model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cuda:0


Running on GPU


In [ ]:

def clean_text(text):
    """Standardize medical text while preserving key clinical terms."""
    if not isinstance(text, str):
        return ""

    # Preserve medical units/measurements
    text = re.sub(
        r"(\d+\s*mg|\d+\s*ml|\d+\s*%|p\s*[<>=]\s*\d+\.\d+)",
        lambda m: m.group().replace(" ", ""),
        text,
    )

    # Remove noise
    text = re.sub(r"[\[\]\{\}\"\'\\]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

In [ ]:
def extract_keywords(texts, top_n=15):
    """Extract meaningful medical keywords using TF-IDF."""
    vectorizer = TfidfVectorizer(
        stop_words=list(MEDICAL_STOP_WORDS), ngram_range=(1, 2), max_features=5000
    )

    try:
        tfidf_matrix = vectorizer.fit_transform(texts)
        scores = np.array(tfidf_matrix.sum(axis=0))[0]
        top_indices = np.argsort(scores)[-top_n:][::-1]
        return [
            term
            for term in vectorizer.get_feature_names_out()[top_indices]
            if not term.isdigit() and len(term) > 2
        ]
    except Exception:
        return []



In [ ]:

def summarize_extractive(texts, max_sentences=5):
    """Select diverse, important sentences using TF-IDF centrality."""
    combined_text = " ".join(texts[:20])  # Process first 20 abstracts max
    sentences = [s for s in sent_tokenize(combined_text) if len(s.split()) > 5]

    if len(sentences) <= max_sentences:
        return combined_text

    vectorizer = TfidfVectorizer(stop_words=list(MEDICAL_STOP_WORDS))
    try:
        sentence_vectors = vectorizer.fit_transform(sentences)
        similarity_matrix = (sentence_vectors * sentence_vectors.T).toarray()
        centrality_scores = np.sum(similarity_matrix, axis=1).flatten()
        top_indices = np.argsort(centrality_scores)[-max_sentences:][::-1]
        return " ".join([sentences[i] for i in sorted(top_indices)])
    except Exception:
        return " ".join(sentences[:max_sentences])



In [ ]:

def summarize_abstractive(text, max_length=SUMMARY_MAX_LENGTH):
    """Generate concise medical summary with BART."""
    if not summarizer or len(text.split()) < 20:
        return text[:max_length]

    try:
        summary = summarizer(
            text, max_length=max_length, min_length=50, do_sample=False, truncation=True
        )[0]["summary_text"]
        return re.sub(r"\s([.,])", r"\1", summary)  # Fix spacing
    except Exception:
        return text[:max_length]



In [ ]:

def identify_topics(abstracts, n_topics=N_TOPICS):
    """Extract clinically relevant topics using LDA."""
    vectorizer = TfidfVectorizer(
        #changed the max_features to 5000 to finetune rouge score
        stop_words=list(MEDICAL_STOP_WORDS), ngram_range=(1, 3), max_features=5000
    )

    try:
        X = vectorizer.fit_transform(abstracts)
        lda = LatentDirichletAllocation(
            n_components=n_topics, random_state=42, learning_method="online"
        )
        lda.fit(X)

        # Get top 10 terms per topic
        terms = []
        for topic in lda.components_:
            top_indices = topic.argsort()[:-11:-1]
            terms.append(
                [
                    term
                    for term in vectorizer.get_feature_names_out()[top_indices]
                    if not term.isdigit()
                ][:5]
            )  # Keep top 5 most relevant

        return lda.transform(X).argmax(axis=1), terms
    except Exception:
        return np.zeros(len(abstracts), dtype=int), []


In [ ]:
def extract_clinical_elements(texts):
    """Extract key clinical components from medical texts with better pattern matching."""
    combined_text = " ".join(texts[:15])  # Analyze first 15 abstracts max

    stats = list(
        set(
            re.findall(
                r"(?:\d+(?:\.\d+)?\s*%|\d+\s*/\s*\d+|p\s*[<>]=?\s*0\.\d+)",
                combined_text,
            )
        )
    )[
        :3
    ]  # Keep top 3 unique

    outcomes = list(
        set(
            re.findall(
                r"(?:associated with|resulted in|led to|caused|improved|reduced|increased|decreased)\s+[\w\s,\-]+?(?:in|among|with|for)\s[\w\s,\-]+",
                combined_text,
                re.IGNORECASE,
            )
        )
    )[:3]

    populations = list(
        set(
            re.findall(
                r"(?:\d+\s*(?:patients|subjects|cases|individuals|participants))\s*(?:with|of|having|diagnosed with|suffering from)\s[\w\s,\-]+",
                combined_text,
                re.IGNORECASE,
            )
        )
    )[:2]

    return {"stats": stats, "outcomes": outcomes, "populations": populations}



In [ ]:

def generate_clinical_summary(topic_terms, clinical_facts):
    """Generate a structured medical summary avoiding prompt leakage."""

    has_populations = bool(clinical_facts["populations"])
    has_outcomes = bool(clinical_facts["outcomes"])
    has_stats = bool(clinical_facts["stats"])

    parts = []

    if has_populations:
        parts.append(f"Key populations: {', '.join(clinical_facts['populations'])}")
    else:
        parts.append(f"Key populations: Study focused on {topic_terms[0]}")

    if has_outcomes:
        parts.append(f"Key outcomes: {', '.join(clinical_facts['outcomes'])}")

    if has_stats:
        parts.append(f"Statistical results: {', '.join(clinical_facts['stats'])}")

    return "\n".join(parts)



In [ ]:
def adjust_max_length(text):
    return min(len(text.split()) // 2, SUMMARY_MAX_LENGTH)

In [ ]:
import nltk
nltk.download("punkt", quiet=True)

True

In [ ]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

dataset = load_dataset("TimSchopf/medical_abstracts", split="train")
df = pd.DataFrame(dataset[:MAX_SAMPLES])
  # Handle column names
abstract_col = next(
        (col for col in ["medical_abstract", "abstract"] if col in df.columns), None)
if not abstract_col:
    raise ValueError("No abstract column found")

df[abstract_col] = df[abstract_col].apply(clean_text).dropna()
print(f"Processing {len(df)} abstracts...")

results = {}
class_labels = (
  sorted(df["condition_label"].unique())
      if "condition_label" in df.columns
      else ["all"]
    )

for class_name in class_labels:
  class_df = (
            df[df["condition_label"] == class_name]
            if "condition_label" in df.columns
            else df
        )
  abstracts = class_df[abstract_col].tolist()
  if not abstracts:
    continue
  topic_labels, topic_terms = identify_topics(abstracts)
  topics = []
  for topic_id, terms in enumerate(topic_terms):
            topic_abstracts = [
                ab for ab, lbl in zip(abstracts, topic_labels) if lbl == topic_id
            ]

            clinical_facts = extract_clinical_elements(topic_abstracts)
            summary = generate_clinical_summary(terms, clinical_facts)

            # Only apply abstractive summarization is alot content
            if len(summary.split()) > 20:
                summary = summarize_abstractive(summary)
            else:
                # Fall back to extractive if not enough  content
                extract_summary = summarize_extractive(topic_abstracts)
                summary = summarize_abstractive(extract_summary)

             #this is new for rouge scores
                print("Topic terms:",topic_terms)
                reference_summary = summarize_extractive(topic_abstracts)
                rouge_scores = compute_rouge(reference_summary, summary)
                topics.append({"id": topic_id + 1, "terms": terms, "summary": summary,"rouge_scores": rouge_scores
})
                results[str(class_name)] = {"topics": topics}
                all_texts = df[abstract_col].tolist()
                results["keywords"] = [
                       kw
                        for kw in extract_keywords(all_texts, top_n=20)
                            if not any(stop in kw for stop in MEDICAL_STOP_WORDS)
    ][
        :15
    ]  # Top 15 most relevant

  with open("medical_summaries_improved.json", "w") as f:
      json.dump(results, f, indent=2)
      print("Processing complete.")


README.md:   0%|          | 0.00/4.96k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.67M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11550 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2888 [00:00<?, ? examples/s]

...

Topic terms: [['facial', 'home', 'facial nerve', 'tumours', 'nerve'], ['spinal', 'lactulose', 'after', 'that', 'mk 801'], ['restenosis', 'pregnancy', 'stroke', 'treated wilson disease', 'epiphyseal'], ['home 43 84', 'slightly', 'cortical n20', '50 100 seven', 'latencies']]
Processing complete.


Your max_length is set to 150, but your input_length is only 68. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


Topic terms: [['splenic', 'artery', 'coronary', 'alcohol', 'sodium'], ['coronary', 'protein', 'filling', 'tricuspid', 'during'], ['glucose', 'graft', 'hypertensive', 'subjects', 'pharmacomechanical'], ['reperfusion', 'blood', 'associations', 'tongue', 'cardioplegic']]
Topic terms: [['splenic', 'artery', 'coronary', 'alcohol', 'sodium'], ['coronary', 'protein', 'filling', 'tricuspid', 'during'], ['glucose', 'graft', 'hypertensive', 'subjects', 'pharmacomechanical'], ['reperfusion', 'blood', 'associations', 'tongue', 'cardioplegic']]
Processing complete.


Your max_length is set to 150, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 150, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


Processing complete.


In [ ]:
print(f"Topic {topic_id + 1} ROUGE Scores: {rouge_scores}")

Topic 4 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.36065573770491804, fmeasure=0.5301204819277109), 'rouge2': Score(precision=0.9767441860465116, recall=0.34710743801652894, fmeasure=0.5121951219512195), 'rougeL': Score(precision=1.0, recall=0.36065573770491804, fmeasure=0.5301204819277109)}


In [ ]:
print("Summary:\n", " ".join(summary.split("\n")))

Summary:
 38 patients with colonic ischemia for comparison with the authors previous experience and with data from the literature. Increased sensitivity and decreased muscarinic receptor number may reflect the cholinergic-induced hypersecretion in nasal allergy but are probably too small to explain the complex allergic reaction.


In [ ]:
extract_summaries = [summarize_extractive([ab]) for ab in abstracts]

# Print first few summaries
for i, summary in enumerate(extract_summaries[:5]):
    print(f"Abstract {i+1} Extractive Summary: {summary}\n")

Abstract 1 Extractive Summary: The aseptically loosened prosthesis provided a means for investigating the in vivo and in vitro activity of the cells associated with the loosening process in seven dogs. The cells were isolated and maintained in culture for sufficient periods of time so that their biologic activity could be studied as well as the effect of different agents added to the cells in vivo or in vitro. The biologic response as determined by interleukin-1 and prostaglandin E2 activity paralleled the roentgenographic appearance of loosening and the technetium images and observations made at the time of revision surgery. The correlation between clinical, roentgenographic, histologic, and biochemical loosening indicates that the canine model is suitable for investigating the mechanisms of prosthetic failure. A canine model permits the study of possible nonsurgical therapeutic interventions with the ultimate hope of stopping or slowing the loosening process.

Abstract 2 Extractive S

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

for i, (abstract, summary) in enumerate(zip(abstracts, extract_summaries[:5])):
    rouge_scores = scorer.score(abstract, summary)
    print(f"Abstract {i+1} ROUGE Scores:", rouge_scores)

Abstract 1 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.8980891719745223, fmeasure=0.9463087248322147), 'rouge2': Score(precision=1.0, recall=0.8974358974358975, fmeasure=0.945945945945946), 'rougeL': Score(precision=1.0, recall=0.8980891719745223, fmeasure=0.9463087248322147)}
Abstract 2 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.5775862068965517, fmeasure=0.73224043715847), 'rouge2': Score(precision=0.9849624060150376, recall=0.5670995670995671, fmeasure=0.7197802197802199), 'rougeL': Score(precision=1.0, recall=0.5775862068965517, fmeasure=0.73224043715847)}
Abstract 3 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.6494845360824743, fmeasure=0.7875000000000001), 'rouge2': Score(precision=0.992, recall=0.6424870466321243, fmeasure=0.7798742138364779), 'rougeL': Score(precision=1.0, recall=0.6494845360824743, fmeasure=0.7875000000000001)}
Abstract 4 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, rec

In [ ]:
with open("extractive_summaries.txt", "w", encoding="utf-8") as file:
    for i, summary in enumerate(extract_summaries, start=1):
        file.write(f"=== Extractive Summary {i} ===\n")  # Title for each summary
        file.write(summary.strip() + "\n")  # Remove extra spaces
        file.write("-" * 50 + "\n\n")  # Separator for readability

In [ ]:
abstractive_summaries = [summarize_abstractive(ab) for ab in abstracts[:5]]

# Print first few summaries
for i, summary in enumerate(abstractive_summaries):
    print(f"Abstract {i+1} Abstractive Summary: {summary}\n")

Your max_length is set to 150, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


Abstract 1 Abstractive Summary: The aseptically loosened prosthesis provided a means for investigating the in vivo and in vitro activity of the cells associated with the loosening process in seven dogs. The correlation between clinical, roentgenographic, histologic, and biochemical loosening indicates that the canine model is suitable to investigate the mechanisms of prosthetic failure.

Abstract 2 Abstractive Summary: In prosthetic or paravalvular prosthetic mitral regurgitation, transthoracic color Doppler flow mapping can sometimes fail to detect the regurgitant jet within the left atrium because of the shadowing by the prosthetic valve. We assessed the utility of color doppler visualization of the flow convergence region (FCR) in 20 consecutive patients. FCR permits localization of the site of the leak with good accuracy.

Abstract 3 Abstractive Summary: Right-sided extracardiac conduits are frequently complicated by obstruction over time. We compared the utility of two-dimensional

In [ ]:
extract_summaries = [summarize_extractive([ab]) for ab in abstracts[:5]]
abstractive_summaries = [summarize_abstractive(es) for es in extract_summaries]

# Print comparisons
for i in range(5):
    print(f"Abstract {i+1} Extractive Summary: {extract_summaries[i]}")
    print(f"Abstract {i+1} Abstractive Summary: {abstractive_summaries[i]}\n")

Your max_length is set to 150, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


Abstract 1 Extractive Summary: The aseptically loosened prosthesis provided a means for investigating the in vivo and in vitro activity of the cells associated with the loosening process in seven dogs. The cells were isolated and maintained in culture for sufficient periods of time so that their biologic activity could be studied as well as the effect of different agents added to the cells in vivo or in vitro. The biologic response as determined by interleukin-1 and prostaglandin E2 activity paralleled the roentgenographic appearance of loosening and the technetium images and observations made at the time of revision surgery. The correlation between clinical, roentgenographic, histologic, and biochemical loosening indicates that the canine model is suitable for investigating the mechanisms of prosthetic failure. A canine model permits the study of possible nonsurgical therapeutic interventions with the ultimate hope of stopping or slowing the loosening process.
Abstract 1 Abstractive S

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

for i, (abstract, summary) in enumerate(zip(abstracts[:5], abstractive_summaries)):
    rouge_scores = scorer.score(abstract, summary)
    print(f"Abstract {i+1} ROUGE Scores:", rouge_scores)

Abstract 1 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.35668789808917195, fmeasure=0.5258215962441315), 'rouge2': Score(precision=0.9818181818181818, recall=0.34615384615384615, fmeasure=0.5118483412322274), 'rougeL': Score(precision=1.0, recall=0.35668789808917195, fmeasure=0.5258215962441315)}
Abstract 2 ROUGE Scores: {'rouge1': Score(precision=0.9701492537313433, recall=0.2801724137931034, fmeasure=0.4347826086956521), 'rouge2': Score(precision=0.9242424242424242, recall=0.26406926406926406, fmeasure=0.4107744107744108), 'rougeL': Score(precision=0.9701492537313433, recall=0.2801724137931034, fmeasure=0.4347826086956521)}
Abstract 3 ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.17525773195876287, fmeasure=0.2982456140350877), 'rouge2': Score(precision=0.9696969696969697, recall=0.16580310880829016, fmeasure=0.28318584070796465), 'rougeL': Score(precision=1.0, recall=0.17525773195876287, fmeasure=0.2982456140350877)}
Abstract 4 ROUGE Scores: {'rouge1': Score(preci

In [ ]:
with open("abstractive_summaries.txt", "w", encoding="utf-8") as file:
    for summary in abstractive_summaries:
        file.write(summary + "\n\n")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
tfidf_matrix = vectorizer.fit_transform(abstracts[:5])

feature_names = vectorizer.get_feature_names_out()
dense = tfidf_matrix.todense().tolist()

# Print highest-ranked keywords for each abstract
for i, row in enumerate(dense):
    sorted_indices = sorted(range(len(row)), key=lambda k: row[k], reverse=True)[:10]
    top_keywords = [feature_names[idx] for idx in sorted_indices]

    print(f"Abstract {i+1} TF-IDF Ranked Keywords:", top_keywords)

Abstract 1 TF-IDF Ranked Keywords: ['loosening', 'activity', 'canine', 'cells', 'model', 'biologic', 'investigating', 'process', 'roentgenographic', 'vitro']
Abstract 2 TF-IDF Ranked Keywords: ['fcr', 'mitral', 'prosthetic', 'regurgitation', 'doppler', 'patients', 'color', 'detected', 'jet', 'regurgitant']
Abstract 3 TF-IDF Ranked Keywords: ['conduits', 'right', 'sided', 'imaging', 'magnetic', 'obstruction', 'resonance', 'doppler', 'echocardiography', 'patients']
Abstract 4 TF-IDF Ranked Keywords: ['bpf', 'fistula', 'bronchopleural', 'closure', 'instillation', 'tetracycline', 'ventilation', 'weaning', 'mechanical', 'air']
Abstract 5 TF-IDF Ranked Keywords: ['shocks', 'arcing', 'coronary', 'sinus', 'dogs', 'nonarcing', 'ablation', 'energy', '40', '18']


In [ ]:
file_path = "extracted_keywords.txt"

with open(file_path, "w", encoding="utf-8") as file:
    for i, row in enumerate(dense):
        sorted_indices = sorted(range(len(row)), key=lambda k: row[k], reverse=True)[:10]
        top_keywords = [feature_names[idx] for idx in sorted_indices]

        file.write(f"Abstract {i+1} Keywords: {', '.join(top_keywords)}\n")

print(f"Extracted keywords saved to {file_path}")

Extracted keywords saved to extracted_keywords.txt
